In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [30]:
# Pull csv training dataset into Pandas dataframe
# Split dataframe into dependent and independent variables
# Normalize x values using min/max scaler
# Add placeholder value of 1 to x values array to account for
# the intercept coefficient
# Split dep and indep dataframes into training and test
training_data_set = pd.read_csv("Default_Fin.csv")
features = ["Bank Balance", "Annual Salary"]

In [41]:
def data_processing(dataFrame, features):
    x_vals = dataFrame[features]
    #x_vals = x_vals.head(9999)
    scaler = StandardScaler().fit(x_vals)
    x_vals = scaler.transform(x_vals)
    x_vals = pd.DataFrame(x_vals, columns=features)
    x_vals.insert(loc=0, column="placeholder", value=np.ones(len(x_vals)))
    y_vals = dataFrame.iloc[:, -1]
    x_vals_train, x_vals_test, y_vals_train, y_vals_test = train_test_split(x_vals, y_vals, random_state=25)
    x_vals_train, x_vals_test, y_vals_train, y_vals_test = x_vals_train.to_numpy(), x_vals_test.to_numpy(), \
                                                           y_vals_train.to_numpy(), y_vals_test.to_numpy()
    return x_vals_train, x_vals_test, y_vals_train, y_vals_test


def get_theta(coef, x):
    theta = 0.0
    for i in range(0, len(x)):
        theta = theta + (coef[i] * x[i])
    return theta


# This function returns the prediction based on the output of
# theta
def get_prediction(theta):
    prediction = 1 / (1 + np.exp(-theta))
    return prediction

# Takes array of coefficients previously used, the dependent
# variable values, the predicted value, the alpha, which
# will default to 0.3, and the array of x values
# This function will calculate the new coefficients and return
# them in the form of an array

def update_coef(coef_array, x_array, y, prediction, alpha):
    new_coef = []
    for i in range(0, len(x_array)):
        coef = coef_array[i] + alpha * (y - prediction) * prediction * (1 - prediction) * x_array[i]
        new_coef.append(coef)
    return new_coef

# Takes array
# Takes an array of x values, an array of coefficients,a "y"
# value and an alpha value, builds the theta value runs theta
# through the sigmoid function and runs that through the
# gradient descent algorithm, update_coef and returns an
# array of coefficients which will be used by the model with
# the non-training data.

def initialize_model(train_row, y, learning_rate):
    # initialize coefficient array with all 0.0
    initial_coef = []
    for i in range(0, len(train_row)):
        initial_coef.append(0.0)
    x_vals = train_row
    y = y
    theta = get_theta(initial_coef, x_vals)
    prediction = get_prediction(theta)
    return update_coef(initial_coef, x_vals, y, prediction, learning_rate)

# Takes independent variables as array, coefficients as array,
# dependent variabls as array, and a learning rate as a float
# Generates theta value using coefficients
def build_model_epoch(independent_variables, coef, dependent_variables, learning_rate):

    coefficients = coef

    for i in range(0, len(independent_variables)):
        theta = get_theta(coefficients, independent_variables[i])
        prediction = get_prediction(theta)
        coefficients = update_coef(coefficients, independent_variables[i], \
                                        dependent_variables[i], prediction, learning_rate)

    return coefficients

def build_model(learning_rate, epoch_count, independent_variables, dependent_variables):
    coefficients = initialize_model(independent_variables[0], dependent_variables[0], learning_rate)
    for i in range(0, epoch_count):
        coefficients = build_model_epoch(independent_variables, coefficients, dependent_variables,
                                              learning_rate)

    return coefficients


def test_model(model_coef, test_inde_vars):
    coef = model_coef
    predictions = []
    for i in range(0, len(test_inde_vars)):
        theta = get_theta(coef, test_inde_vars[i])
        prediction = get_prediction(theta)
        if prediction < 0.5:
            predictions.append(0)
        else:
            predictions.append(1)
    return predictions

def score_test(test, y_vals_test):
    one_is_one = 0
    zero_is_zero = 0
    one_is_zero = 0
    zero_is_one = 0

    for i in range(0, len(test)):
        if test[i] == 1 and y_vals_test[i] == 1:
            one_is_one += 1
        elif test[i] == 0 and y_vals_test[i] == 0:
            zero_is_zero += 1
        elif test[i] == 0 and y_vals_test[i] == 1:
            one_is_zero += 1
        else:
            zero_is_one += 1
    print(one_is_one, zero_is_zero, one_is_zero, zero_is_one)


In [43]:
x_train, x_test, y_train, y_test = data_processing(training_data_set, features)
model = build_model( 0.3, 5, x_train, y_train)
test = test_model(model, x_test)
score_test(test, y_test)

31 2400 51 18
